In [ ]:
import os
import pydicom
import glob
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
from pathlib import Path
from ipywidgets import interact, interactive

In [ ]:
def load_scan(path):
    slices = [pydicom.dcmread(path + '/' + s) for s in os.listdir(path)]
    slices.sort(key = lambda x: int(x.InstanceNumber))
    try:
        slice_thickness = np.abs(slices[0].ImagePositionPatient[2] - slices[1].ImagePositionPatient[2])
    except:
        slice_thickness = np.abs(slices[0].SliceLocation - slices[1].SliceLocation)
        
    for s in slices:
        s.SliceThickness = slice_thickness
        
    return slices

In [ ]:
def get_pixels_hu(scans):
    image = np.stack([s.pixel_array for s in scans])
    # Convert to int16 (from sometimes int16), 
    # should be possible as values should always be low enough (<32k)
    image = image.astype(np.int16)

    # Set outside-of-scan pixels to 1
    # The intercept is usually -1024, so air is approximately 0
    image[image == -2000] = 0
    
    # Convert to Hounsfield units (HU)
    intercept = scans[0].RescaleIntercept
    slope = scans[0].RescaleSlope
    
    if slope != 1:
        image = slope * image.astype(np.float64)
        image = image.astype(np.int16)
        
    image += np.int16(intercept)
    
    return np.array(image, dtype=np.int16)

In [ ]:
data_path = '../data/example_5/'
patient = load_scan(data_path)
imgs = get_pixels_hu(patient)

In [ ]:
plt.hist(imgs.flatten(), bins=50, color='c')
plt.xlabel("Hounsfield Units (HU)")
plt.ylabel("Frequency")
plt.show()

In [ ]:
def sample_stack(stack, rows=6, cols=6, start_with=10, show_every=5):
    fig,ax = plt.subplots(rows,cols,figsize=[12,12],dpi=100)
    for i in range(rows*cols):
        ind = start_with + i*show_every
        ax[int(i/rows),int(i % rows)].set_title('slice %d' % ind)
        ax[int(i/rows),int(i % rows)].imshow(stack[ind],cmap='gray')
        ax[int(i/rows),int(i % rows)].axis('off')
    plt.show()

sample_stack(imgs)

In [ ]:
imgs.shape

In [ ]:
imgs[0]

In [ ]:
type(imgs[0])

In [ ]:
plt.imshow(imgs[150],cmap="bone")

In [ ]:
a = imgs[150].copy()

In [ ]:
plt.imshow(a, cmap="bone")
MAX_HU_LUNG= -380.0
MIN_HU_LUNG= -1500.0
a[a<=MIN_HU_LUNG] = 0
a[a>=MAX_HU_LUNG] = 0
plt.imshow(a, cmap="bone")

In [ ]:
from skimage.filters.rank import mean
from skimage.morphology import disk
plt.imshow(mean(a, disk(10)), cmap="bone")

In [ ]:
plt.hist(a.flatten(), bins=50, color='c')
plt.xlabel("Hounsfield Units (HU)")
plt.ylabel("Frequency")
plt.show()

In [ ]:
print("Slice Thickness: %f" % patient[0].SliceThickness)
print("Pixel Spacing (row, col): (%f, %f) " % (patient[0].PixelSpacing[0], patient[0].PixelSpacing[1]))